# SNP Filtering and Analysis

Original files of new and known SNPs from GWAS papers are filtered to remove SNPs in linkage disequilibrium with other SNPs in the lists. Analysis is then done on the filtered new SNPs as well as GWAS Catalog SNPs to determine significance of nervous system disease related SNPs and autoimmune disease related SNPs being present in brain and blood genes vs a uniform distribution.

# Loading SNP Information from Files

Initially, data structures are set up to store information about the SNPs read from the input files.

Essential Python libraries are imported.

In [1]:
import csv
import collections as col
import numpy as np
import copy

Data structures to store SNP information are initialized.

In [2]:
# Sets of new and known SNPs associated with each paper and phenotype
# papersnps[paperid][pheno]["new"] and papersnps[paperid][pheno]["known"] to access sets of SNPs
papersnps = col.defaultdict(lambda: col.defaultdict(lambda: col.defaultdict(set)))

# Sets of paper to snps for known associations
knownpapersnps = col.defaultdict(set)

# Dictionary of the phenotypes of known snps to use if GWAS catalog doesn't have for a snp
knownphenos = {}

# For each paper, the pvalue of every SNP for both new and known snps
newpvals = col.defaultdict(dict)
knownpvals = col.defaultdict(dict)

# Information about SNPs including chromosome and physical basepair location
# snpsinfo[snpid]["chr"], snpsinfo[snpid]["bp"] for each value
snpsinfo = col.defaultdict(dict)

# For each chromosome, maps from basepair location to genetic location
# genloc[chr][bp] to access the genetic location for that basepair location of that chromosome
genloc = col.defaultdict(dict)

# All original new and known snps stored by each paper
orignewassoc = col.defaultdict(set)
origknownassoc = col.defaultdict(set)

New and known SNP associations and information from original input files "associations.new.tsv" and "associations.known.tsv" are read and stored in corresponding data structures. These files are obtained from https://github.com/kuleshov/gwaskb/tree/master/notebooks/results.

In [3]:
# Parsing through associations.new.tsv for information
with open("associations.new.tsv") as new:
    for line in csv.reader(new, delimiter="\t"):
        paperid = line[0]
        snpid = line[1]
        if snpid[0] == "R":
            snpid = snpid.replace("R", "r")
        pheno1 = line[2]
        pheno2 = line[3]
        pval = float(line[4])
        papersnps[paperid][(pheno1, pheno2)]["new"].add(snpid)
        orignewassoc[paperid].add((paperid, snpid, pheno1, pheno2, pval))
        newpvals[paperid][snpid] = pval

# Parsing through associations.known.tsv for information
with open("associations.known.tsv") as known:
    for line in csv.reader(known, delimiter="\t"):
        paperid = line[0]
        snpid = line[1]
        if snpid[0] == "R":
            snpid = snpid.replace("R", "r")
        pheno1 = line[2]
        pheno2 = line[3]
        pval = float(line[4])
        papersnps[paperid][(pheno1, pheno2)]["known"].add(snpid)
        origknownassoc[paperid].add((paperid, snpid, pheno1, pheno2, pval))
        knownpapersnps[paperid].add(snpid)
        knownphenos[snpid] = pheno1 + " " + pheno2
        knownpvals[paperid][snpid] = pval

Additional chromosome and basepair location information about the new and known SNPs are read from the input files "newbps.txt" and "knownbps.txt". These files are obtained from querying lists of new and known SNPs at https://bioinformatics.uef.fi/varietas/index.php?about=yes which uses the NCBI database using the Genome Version GRCh37.

In [4]:
# Parsing through newbps.txt for information
with open("newbps.txt") as new2:
    for line in csv.reader(new2, delimiter="\t"):
        snpid = line[0].strip()
        chrom = "chr" + line[2].strip()
        bp = int(line[3].strip())
        snpsinfo[snpid]["chr"] = chrom
        snpsinfo[snpid]["bp"] = bp

# Parsing through knownbps.txt for information
with open("knownbps.txt") as known2:
    for line in csv.reader(known2, delimiter="\t"):
        snpid = line[0].strip()
        chrom = "chr" + line[2].strip()
        bp = int(line[3].strip())
        snpsinfo[snpid]["chr"] = chrom
        snpsinfo[snpid]["bp"] = bp

Files containing a genetic map that maps basepair location to genetic location for 22 chromosomes are read and stored. These genetic map files are obtained from http://www.shapeit.fr/pages/m02_formats/geneticmap.html and are located in the "genetic_map_b37" folder. The Genome Version is GRCh37.

In [5]:
# Parses through all genetic map files for the chromosomes to get genetic location information and creates lists of bps for each chromosome
numchroms = 22

chrbps = {}

for i in range(1, numchroms + 1):
    chromosome = "chr" + str(i)
    file = "genetic_map_b37/genetic_map_" + chromosome + "_combined_b37.txt"
    with open(file) as genmap:
        for line in csv.reader(genmap, delimiter=" "):
            bp = int(line[0])
            gen = float(line[2])
            genloc[chromosome][bp] = gen
    chrbps[chromosome] = genloc[chromosome].keys()

Additionally, a file containing all known phenotypes for known SNPs in GWAS Catalog is read and parsed for SNP-phenotype information. This is used for replacing the known SNP phenotypes with a more accurate phenotype from GWAS Catalog if it exists. The file is named "gwas_catalog_v1.0-associations_e87_r2017-02-13.tsv" and is obtained from https://www.ebi.ac.uk/gwas/docs/file-downloads under "All associations".

In [6]:
# Dictionary to store GWAS Catalog SNP phenotypes
gwasphenos = {}

# Parsing through GWAS catalog for updated known phenotype
with open("gwas_catalog_v1.0-associations_e87_r2017-02-13.tsv") as new:
    for line in csv.reader(new, delimiter="\t"):
        snpid = line[16]
        pheno = line[7]
        gwasphenos[snpid] = pheno

# Filtering New and Known SNPs for Associations

Here, the original new SNPs and known SNPs lists are filtered to remove SNPs that are in linkage disequilibrium (LD) with other SNPs from the lists. From the new SNPs, SNPs in LD with known SNPs were removed. Additionally, for any pair of new SNPs in LD with each other, the SNP with the lower p-value was removed. Similarly, from the known SNPs, for any pair of known SNPs in LD with each other, the SNP with the lower p-value was removed. Linkage disequilibrium between two SNPs was defined as being on the same chromosome and having a genetic distance of less than or equal to 0.1 centiMorgans (cM) between the two SNPs. To do this, pairs of new-new, new-known, and known-known SNPs were examined with each other and if they were on the same chromosome and within 100000 basepairs (physical distance) of each other, their genetic distance was computed. If the genetic distance was less than the threshold of 0.1 cM, the association between the two SNPs was documented as being in LD.

Initial basepair threshold and list of SNPs in our database was created.

In [7]:
bpthreshold = 100000 # Basepair distance threshold for comparing SNPs

gendistancethreshold = 0.1 # Genetic distance threshold for filtering out of original lists

keys = snpsinfo.keys()

A function to obtain the genetic location of a SNP was initialized. The function works by taking in the chromsome and basepair location of the SNP and looks through the genetic map for the closest values less than or equal to and greater than or equal to the basepair location given for which a genetic location exists. From these two values, the genetic location of the SNP is interpolated.

In [8]:
# Gets genetic location of a SNP given chromosome and basepair location
def getGenLoc(chrom, bp):
    if chrom == "chrX":  # no genetic location info for X chromosome in genetic map
        return -1
    chrombps = chrbps[chrom]
    if len(chrombps) <= 0:
        return -1
    greaterarr = [val for val in chrombps if val >= bp]
    lessarr = [val for val in chrombps if val <= bp]
    if len(greaterarr) == 0 or len(lessarr) == 0:
        return -1
    greater = min(greaterarr)
    less = max(lessarr)
    x = [less, greater]
    y = [genloc[chrom][less], genloc[chrom][greater]]
    return np.interp(bp, x, y)

LD associations between new SNPs and known SNPs are found and stored. Basically, new and known SNPs from the same paper and consisting of the same phenotype description are compared. If the SNPs are found on the same chromosome and within 100000 basepairs of each other in terms of physical distance, their genetic distance is calculated. If this genetic distance is within 0.1 cM, the SNPs are said to be in LD with each other and the association is recorded.

In [9]:
# Contains associations for every paper in a list of tuples: (new SNPid, known SNPid, physical distance (bp), genetic distance (cM))
# associations[paper] is a list of tuples of the associations for each paper
associations = col.defaultdict(list)

# For new - known associations
for paper in papersnps:
    for pheno in papersnps[paper]:
        for newsnp in papersnps[paper][pheno]["new"]:
            if not (newsnp in keys):
                continue
            if not ("chr" in snpsinfo[newsnp].keys()):
                continue
            newsnpchr = snpsinfo[newsnp]["chr"]
            newsnpbp = snpsinfo[newsnp]["bp"]
            for knownsnp in papersnps[paper][pheno]["known"]:
                if not (knownsnp in keys):
                    continue
                if not ("chr" in snpsinfo[knownsnp].keys()):
                    continue
                knownsnpchr = snpsinfo[knownsnp]["chr"]
                knownsnpbp = snpsinfo[knownsnp]["bp"]
                bpdistance = abs(newsnpbp - knownsnpbp)
                if newsnpchr == knownsnpchr and bpdistance <= bpthreshold:
                    newsnpgen = getGenLoc(newsnpchr, newsnpbp)
                    knownsnpgen = getGenLoc(knownsnpchr, knownsnpbp)
                    if newsnpgen == -1 or knownsnpgen == -1:
                        continue
                    gendistance = abs(newsnpgen - knownsnpgen)
                    snppheno = pheno[0] + " " + pheno[1]
                    associations[paper].append((snppheno, newsnp, knownsnp, bpdistance, gendistance))

These new-known SNP associations are then written to a .txt file named "associationsnk.txt" and the original new SNPs list is filtered to exclude new SNPs in LD with known SNPs.

In [10]:
# Writes new - known associations to txt file
output = open("associationsnk.txt", "w")

output.write("Paper ID, Phenotype, New SNP ID, Known SNP ID, Physical Distance (bp), Genetic Distance (cM)\n")

filternk = col.defaultdict(set)

for paper in associations:
    for assoc in associations[paper]:
        output.write(paper + " " + assoc[0] + " " + assoc[1] + " " + assoc[2] + " " + str(assoc[3]) + " " + str(assoc[4]) + "\n")
        if assoc[4] <= gendistancethreshold:
            filternk[paper].add(assoc[1])

output.close()

# Filters original new list by removing new snps with new - known associations
filterednewlist = col.defaultdict(list)

for paper in orignewassoc:
    papervals = list(orignewassoc[paper])
    assocsnps = list(filternk[paper])
    for val in papervals:
        snpid = val[1]
        if snpid not in assocsnps:
            filterednewlist[paper].append(val)

Similarly, LD associations between known SNPs and other known SNPs are found and stored. The GWAS phenotypes for the known SNPs are used if they exist.

In [11]:
# For known - known associations using GWAS Phenotypes
gwassnps = gwasphenos.keys()

knownassociations = col.defaultdict(list)

for paper in knownpapersnps:
    snpslist = list(knownpapersnps[paper])
    for i in range(len(snpslist)):
        knownsnp1 = snpslist[i]
        usegwas = False
        if knownsnp1 in gwassnps:
            knownsnppheno1 = gwasphenos[knownsnp1]
            usegwas = True
        else:
            knownsnppheno1 = knownphenos[knownsnp1]
        if not (knownsnp1 in keys):
            continue
        if not ("chr" in snpsinfo[knownsnp1].keys()):
            continue
        knownsnpchr1 = snpsinfo[knownsnp1]["chr"]
        knownsnpbp1 = snpsinfo[knownsnp1]["bp"]
        for j in range(i, len(snpslist)):
            knownsnp2 = snpslist[j]
            if knownsnp2 in gwassnps and usegwas:
                knownsnppheno2 = gwasphenos[knownsnp2]
            else:
                knownsnppheno2 = knownphenos[knownsnp2]
            if not (knownsnp2 in keys) or knownsnp1 == knownsnp2 or knownsnppheno1 != knownsnppheno2:
                continue
            if not ("chr" in snpsinfo[knownsnp2].keys()):
                continue
            knownsnpchr2 = snpsinfo[knownsnp2]["chr"]
            knownsnpbp2 = snpsinfo[knownsnp2]["bp"]
            bpdistance = abs(knownsnpbp1 - knownsnpbp2)
            if knownsnpchr1 == knownsnpchr2 and bpdistance <= bpthreshold:
                knownsnpgen1 = getGenLoc(knownsnpchr1, knownsnpbp1)
                knownsnpgen2 = getGenLoc(knownsnpchr2, knownsnpbp2)
                if knownsnpgen1 == -1 or knownsnpgen2 == -1:
                    continue
                gendistance = abs(knownsnpgen1 - knownsnpgen2)
                snppheno = knownsnppheno1
                knownassociations[paper].append((snppheno, knownsnp1, knownsnp2, bpdistance, gendistance))

These known-known SNP associations are then written to a .txt file named "associationskk.txt" and the original known SNPs list is also filtered. For each pair of known SNPs in LD with each other, the known SNP with the lower p-value is removed from the list.

In [12]:
# Writes known - known associations to txt file
output = open("associationskk.txt", "w")

output.write("Paper ID, Phenotype, Known SNP 1 ID, Known SNP 2 ID, Physical Distance (bp), Genetic Distance (cM)\n")

filterkk = col.defaultdict(set)

for paper in knownassociations:
    for assoc in knownassociations[paper]:
        output.write(paper + " " + assoc[0] + " " + assoc[1] + " " + assoc[2] + " " + str(assoc[3]) + " " + str(assoc[4]) + "\n")
        if assoc[4] <= gendistancethreshold:
            pval1 = knownpvals[paper][assoc[1]]
            pval2 = knownpvals[paper][assoc[2]]
            if pval1 < pval2:
                filterkk[paper].add(assoc[1])
            else:
                filterkk[paper].add(assoc[2])

output.close()

# Filters original known list by removing known snps with known - known association and the lower pvalue of the pair
filteredknownlist = col.defaultdict(list)

for paper in origknownassoc:
    papervals = list(origknownassoc[paper])
    assocsnps = list(filterkk[paper])
    for val in papervals:
        snpid = val[1]
        if snpid not in assocsnps:
            filteredknownlist[paper].append(val)

LD associations between new SNPs and other new SNPs are also found and stored.

In [18]:
# For new - new associations
newassociations = col.defaultdict(list)

for paper in papersnps:
    for pheno in papersnps[paper]:
        snpslist = list(papersnps[paper][pheno]["new"])
        for i in range(len(snpslist)):
            newsnp1 = snpslist[i]
            if not (newsnp1 in keys):
                continue
            if not ("chr" in snpsinfo[newsnp1].keys()):
                continue
            newsnpchr1 = snpsinfo[newsnp1]["chr"]
            newsnpbp1 = snpsinfo[newsnp1]["bp"]
            for j in range(i, len(snpslist)):
                newsnp2 = snpslist[j]
                if not (newsnp2 in keys) or newsnp1 == newsnp2:
                    continue
                if not ("chr" in snpsinfo[newsnp2].keys()):
                    continue
                newsnpchr2 = snpsinfo[newsnp2]["chr"]
                newsnpbp2 = snpsinfo[newsnp2]["bp"]
                bpdistance = abs(newsnpbp1 - newsnpbp2)
                if newsnpchr1 == newsnpchr2 and bpdistance <= bpthreshold:
                    newsnpgen1 = getGenLoc(newsnpchr1, newsnpbp1)
                    newsnpgen2 = getGenLoc(newsnpchr2, newsnpbp2)
                    if newsnpgen1 == -1 or newsnpgen2 == -1:
                        continue
                    gendistance = abs(newsnpgen1 - newsnpgen2)
                    snppheno = pheno[0] + " " + pheno[1]
                    newassociations[paper].append((snppheno, newsnp1, newsnp2, bpdistance, gendistance))

These new-new SNP associations are then written to a .txt file named "associationsnn.txt". Here, a copy of the filtered new SNPs list (from filtering out new-known associations) is made so that there can be two filtered new SNPs lists: one list with just filtered new-known associations and one list with both filtered new-known associations and filtered new-new associations. To filter new-new associations, for each pair of new SNPs in LD with each other, the new SNP with the lower p-value is removed from the list.

In [19]:
# Writes new - new associations to txt file
output = open("associationsnn.txt", "w")

output.write("Paper ID, Phenotype, New SNP 1 ID, New SNP 2 ID, Physical Distance (bp), Genetic Distance (cM)\n")

filternn = col.defaultdict(set)

for paper in newassociations:
    for assoc in newassociations[paper]:
        output.write(paper + " " + assoc[0] + " " + assoc[1] + " " + assoc[2] + " " + str(assoc[3]) + " " + str(assoc[4]) + "\n")
        if assoc[4] <= gendistancethreshold:
            pval1 = newpvals[paper][assoc[1]]
            pval2 = newpvals[paper][assoc[2]]
            if pval1 < pval2:
                filternn[paper].add(assoc[1])
            else:
                filternn[paper].add(assoc[2])

output.close()

# Copies filterednewlist to keep new - new associations
filterednewlist2 = copy.deepcopy(filterednewlist)

# Filters filterednewlist by removing new snps with new - new associations
for paper in filterednewlist:
    papervals = filterednewlist[paper]
    assocsnps = list(filternn[paper])
    for val in papervals:
        snpid = val[1]
        if snpid in assocsnps:
            filterednewlist[paper].remove(val)

Finally, the filtered lists of new and known SNPs are written to tsv files. "filtered.associations.new.tsv" is the list of new SNPs with filtered out new-new associations as well as new-known associations, "filtered.associations.new2.tsv" is the list of new SNPs with filtered out new-known associations, and "filtered.associations.known.tsv" is the list of known SNPs with filtered out known-known associations.

In [20]:
# Create filtered new list and filtered known list files

# List with filtered out new - new associations
output = open("filtered.associations.new.tsv", "w")

for paper in filterednewlist:
    for val in filterednewlist[paper]:
        output.write(paper + "\t" + val[1] + "\t" + val[2] + "\t" + val[3] + "\t" + str(val[4]) + "\n")

output.close()

# List with kept new - new associations
output = open("filtered.associations.new2.tsv", "w")

for paper in filterednewlist2:
    for val in filterednewlist2[paper]:
        output.write(paper + "\t" + val[1] + "\t" + val[2] + "\t" + val[3] + "\t" + str(val[4]) + "\n")

output.close()


output = open("filtered.associations.known.tsv", "w")

for paper in filteredknownlist:
    for val in filteredknownlist[paper]:
        output.write(paper + "\t" + val[1] + "\t" + val[2] + "\t" + val[3] + "\t" + str(val[4]) + "\n")

output.close()

In [21]:
print "Filtering SNPs Complete"

Filtering SNPs Complete


# Finding SNP - Gene Correlations for Filtered New SNPs

Statistical analysis is performed on the filtered new SNPs to find whether SNPs relating to specific disease groups are more enriched in certain types of genes. Specifically, we want to see if nervous system related (NS) SNPs are more enriched in brain-related genes and if autoimmune disease related (AU) SNPs are more enriched in blood-related genes.

Additional data structures are initialized to store information about the disease related SNPs and the genes.

In [41]:
# Traits relating to Nervous System and Autoimmune diseases
nstraits = []
autraits = []

# SNPS from filtered new SNPs list corresponding to NS or AU
nssnps = set([])
ausnps = set([])

# Information about basepair locations of brain and blood related genes
# braingenes[chr] gives list of (start, end) basepair locations for blood related genes of that chromosome
braingenes = col.defaultdict(list)
bloodgenes = col.defaultdict(list)

Lists of filtered new SNP traits associated with nervous system (NS) and autoimmune (AU) diseases are read and stored. These files are "nstraitslist.tsv" and "autraitslist.tsv" and the traits are obtained by manual selection of phenotypes from the "filtered.associations.new2.tsv" SNP phenotypes that correspond with NS and AU diseases.

In [42]:
with open("nstraitslist.tsv") as new:
    for line in csv.reader(new, delimiter="\t"):
        nstraits.append(line[0])

with open("autraitslist.tsv") as new:
    for line in csv.reader(new, delimiter="\t"):
        autraits.append(line[0])

The filtered new SNPs list is read and corresponding NS and AU SNPs are stored. By default, the filtered new SNPs list used is "filtered.associations.new2.tsv" which does not filter out new-new SNP associations but the analysis can be done with the "filtered.associations.new.tsv" list which does filter out new-new SNP associations by replacing the file name "filtered.associations.new2.tsv" with "filtered.associations.new.tsv" as shown in the code. The total numbers of NS SNPs and AU SNPs are printed out after the SNPs are read and stored.

In [43]:
# By default, uses "filtered.associations.new2.tsv" which is the filtered new SNPs list without removing new-new associations
with open("filtered.associations.new2.tsv") as new:
    for line in csv.reader(new, delimiter="\t"):
        if line[2] in nstraits:
            nssnps.add(line[1])
        elif line[2] in autraits:
            ausnps.add(line[1])

nssnps = list(nssnps)
ausnps = list(ausnps)

snpids = snpsinfo.keys()

for snp in nssnps:
    if snp not in snpids:
        nssnps.remove(snp)

for snp in ausnps:
    if snp not in snpids:
        ausnps.remove(snp)

numnssnps = len(nssnps)
numausnps = len(ausnps)

print "Number of NS SNPS:", numnssnps
print "Number of AU SNPS:", numausnps

Number of NS SNPS: 283
Number of AU SNPS: 155


Information regarding the basepair locations/windows of brain and blood related genes are read and stored from files named "output_gtexbraingenes.bed" and "output_gtexbloodgenes.bed". To obtain these brain and blood related gene files, brain and blood genes are selected by filtering all genes stored in GTEx Portal (https://gtexportal.org/home/datasets) by tissue type (brain and blood related tissues) and a mean RPKM threshold. After these lists of brain and blood related genes are created, the lists are passed into the NCBI database query (https://www.ncbi.nlm.nih.gov/sites/batchentrez) to obtain the basepair locations of these genes. However, since the NCBI query references using Genome version GRCh38, the gene information is converted to a corresponding BED input format to convert the locations from GRCh38 to GRCh37 using the Ensembl Assembly Converter (http://grch37.ensembl.org/Homo_sapiens/Tools/AssemblyConverter?db=core) which gives us our final gene files of "output_gtexbraingenes.bed" and "output_gtexbloodgenes.bed".

In [44]:
# Using Genome version 37
with open("output_gtexbraingenes.bed") as new:
    for line in csv.reader(new, delimiter="\t"):
        if len(line[0]) <= 2:
            braingenes["chr" + line[0]].append((int(line[1]), int(line[2])))

with open("output_gtexbloodgenes.bed") as new:
    for line in csv.reader(new, delimiter="\t"):
        if len(line[0]) <= 2:
            bloodgenes["chr" + line[0]].append((int(line[1]), int(line[2])))

Now we can calculate which fractions of NS and AU SNPs are found in brain and blood genes. A function that returns whether a SNP location is within a certain basepair window threshold of a gene basepair window is initialized.

In [45]:
bpwindowthreshold = 200000

def inRange(snpbp, startbp, endbp):
    return snpbp >= startbp - bpwindowthreshold and snpbp <= endbp + bpwindowthreshold

The percentages of NS SNPs found in brain genes, blood genes, both brain and blood genes, and neither brain nor blood genes are calculated and printed.

In [46]:
# For NS SNPs
nsbrain = 0
nsblood = 0
nsboth = 0
nsnone = 0

for snp in nssnps:
    snpchr = snpsinfo[snp]["chr"]
    snpbp = snpsinfo[snp]["bp"]

    inbrain = 0
    for start, end in braingenes[snpchr]:
        if inRange(snpbp, start, end):
            inbrain = 1
            break

    inblood = 0
    for start, end in bloodgenes[snpchr]:
        if inRange(snpbp, start, end):
            inblood = 1
            break

    nsbrain += inbrain
    nsblood += inblood

    inboth = inbrain + inblood
    if inboth == 2:
        nsboth += 1
        nsbrain -= 1
        nsblood -= 1
    elif inboth == 0:
        nsnone += 1


print "Percentage of NS SNPs Found in Brain Genes:", float(nsbrain) / numnssnps
print "Percentage of NS SNPs Found in Blood Genes:", float(nsblood) / numnssnps
print "Percentage of NS SNPs Found in Both:", float(nsboth) / numnssnps
print "Percentage of NS SNPs Found in Neither:", float(nsnone) / numnssnps

Percentage of NS SNPs Found in Brain Genes: 0.0883392226148
Percentage of NS SNPs Found in Blood Genes: 0.0247349823322
Percentage of NS SNPs Found in Both: 0.0777385159011
Percentage of NS SNPs Found in Neither: 0.809187279152


Similarly, the percentages of AU SNPs found in brain genes, blood genes, both brain and blood genes, and neither brain nor blood genes are calculated and printed.

In [47]:
# For AU SNPs
aubrain = 0
aublood = 0
auboth = 0
aunone = 0

for snp in ausnps:
    snpchr = snpsinfo[snp]["chr"]
    snpbp = snpsinfo[snp]["bp"]

    inbrain = 0
    for start, end in braingenes[snpchr]:
        if inRange(snpbp, start, end):
            inbrain = 1
            break

    inblood = 0
    for start, end in bloodgenes[snpchr]:
        if inRange(snpbp, start, end):
            inblood = 1
            break

    aubrain += inbrain
    aublood += inblood

    inboth = inbrain + inblood
    if inboth == 2:
        auboth += 1
        aubrain -= 1
        aublood -= 1
    elif inboth == 0:
        aunone += 1


print "Percentage of AU SNPs Found in Brain Genes:", float(aubrain) / numausnps
print "Percentage of AU SNPs Found in Blood Genes:", float(aublood) / numausnps
print "Percentage of AU SNPs Found in Both:", float(auboth) / numausnps
print "Percentage of AU SNPs Found in Neither:", float(aunone) / numausnps

Percentage of AU SNPs Found in Brain Genes: 0.0322580645161
Percentage of AU SNPs Found in Blood Genes: 0.0645161290323
Percentage of AU SNPs Found in Both: 0.129032258065
Percentage of AU SNPs Found in Neither: 0.774193548387


The specific number of NS and AU SNPs in brain, blood, both, and neither are also printed for reference.

In [48]:
print "NS SNPs in Brain:", nsbrain
print "NS SNPs in Blood:", nsblood
print "AU SNPs in Brain:", aubrain
print "AU SNPs in Blood:", aublood
print "NS Both:", nsboth
print "NS None:", nsnone
print "AU Both", auboth
print "AU None:", aunone

NS SNPs in Brain: 25
NS SNPs in Blood: 7
AU SNPs in Brain: 5
AU SNPs in Blood: 10
NS Both: 22
NS None: 229
AU Both 20
AU None: 120


We now want to perform a Chi-Square test to see if it is indeed significant that NS and AU related SNPs are found more in brain or blood genes vs a uniform distribution of SNPs in genes. The Chi-Square test we use will contain 2 degrees of freedom (# of rows * # of columns) as we have 2 rows (NS and AU SNPs) and 3 columns (being in a brain related gene, being in a blood related gene, or other which combines being in both brain and blood genes and being in neither brain or blood genes) which gives us a significance threshold of 5.99 at a P-value of 0.05.

In [50]:
# Three columns: brain, blood, other
def chiSquare(nsbrain, nsblood, aubrain, aublood, nsother, auother):
    blood = nsblood + aublood
    brain = nsbrain + aubrain
    other = nsother + auother
    total = brain + blood + other
    ns = nsbrain + nsblood + nsother
    au = aubrain + aublood + auother

    nsbr = (float(brain) / total) * ns
    nsbl = (float(blood) / total) * ns
    aubr = (float(brain) / total) * au
    aubl = (float(blood) / total) * au
    nsot = (float(other) / total) * ns
    auot = (float(other) / total) * au

    chisquareval = ((nsbrain - nsbr)**2 / nsbr) + ((nsblood - nsbl)**2 / nsbl) + ((aubrain - aubr)**2 / aubr) + ((aublood - aubl)**2 / aubl) + ((nsother - nsot)**2 /nsot) + ((auother - auot)**2 /auot)

    return chisquareval

# 95% confidence threshold for 2 degrees of freedom
chithreshold = 5.99

The Chi-Square value and significance threshold are printed out. If the Chi-Square value is greater than or equal to the significance threshold, then it is indeed significant that NS and AU related SNPs are found more in brain or blood genes than a uniform distribution.

In [51]:
nsother = nsboth + nsnone
auother = auboth + aunone

chithreshold = 5.99
chisquareval = chiSquare(nsbrain, nsblood, aubrain, aublood, nsother, auother)

print "Chi Square Value:", chisquareval
print "Threshold for Significance:", chithreshold

Chi Square Value: 8.7118795957
Threshold for Significance: 5.99


# Finding SNP - Gene Correlations for GWAS Catalog SNPs

Similarly, an analysis of NS and AU SNPs correlated with brain and blood genes is done on NS and AU SNPs from the GWAS Catalog to see if the significance is also prevalent in general NS and AU SNPs. The GWAS Catalog SNP files are found below in the specified lists.

The GWAS Catalog SNP files are read and stored. The total numbers of GWAS Catalog NS and AU SNPs are printed.

In [52]:
# Run Analysis on GWAS Catalog NS and AU SNPs

gwasnsfiles = ["gwassclerosis.tsv", "gwasalzheimers.tsv", "gwasparkinsons.tsv", "gwasepilepsy.tsv", "gwasstroke.tsv", "gwasaphasia.tsv"]
gwasaufiles = ["gwasrheumatoid.tsv", "gwaslupus.tsv", "gwasibd.tsv", "gwasdiabetes.tsv", "gwaspsorias.tsv"]

gwassnpsinfo = col.defaultdict(dict)
gwasnssnps = []
gwasausnps = []

for filename in gwasnsfiles:
    with open(filename) as new:
        for line in csv.reader(new, delimiter="\t"):
            if line[21] == "" or not line[11].isdigit() or not line[12].isdigit():
                continue
            gwassnp = line[21]
            gwaschr = "chr" + line[11]
            gwasbp = int(line[12])
            gwassnpsinfo[gwassnp]["chr"] = gwaschr
            gwassnpsinfo[gwassnp]["bp"] = gwasbp
            gwasnssnps.append(gwassnp)

for filename in gwasaufiles:
    with open(filename) as new:
        for line in csv.reader(new, delimiter="\t"):
            if line[21] == "" or not line[11].isdigit() or not line[12].isdigit():
                continue
            gwassnp = line[21]
            gwaschr = "chr" + line[11]
            gwasbp = int(line[12])
            gwassnpsinfo[gwassnp]["chr"] = gwaschr
            gwassnpsinfo[gwassnp]["bp"] = gwasbp
            gwasausnps.append(gwassnp)


gnumnssnps = len(gwasnssnps)
gnumausnps = len(gwasausnps)

print "GWAS Catalog NS SNPS:", gnumnssnps
print "GWAS Catalog AU SNPS:", gnumausnps

GWAS Catalog NS SNPS: 1805
GWAS Catalog AU SNPS: 2956


The percentages of GWAS Catalog NS SNPs found in brain genes, blood genes, both brain and blood genes, and neither brain nor blood genes are calculated and printed.

In [53]:
# For GWAS Catalog NS SNPs
gnsbrain = 0
gnsblood = 0
gnsboth = 0
gnsnone = 0

for snp in gwasnssnps:
    snpchr = gwassnpsinfo[snp]["chr"]
    snpbp = gwassnpsinfo[snp]["bp"]

    inbrain = 0
    for start, end in braingenes[snpchr]:
        if inRange(snpbp, start, end):
            inbrain = 1
            break

    inblood = 0
    for start, end in bloodgenes[snpchr]:
        if inRange(snpbp, start, end):
            inblood = 1
            break

    gnsbrain += inbrain
    gnsblood += inblood

    inboth = inbrain + inblood
    if inboth == 2:
        gnsboth += 1
        gnsbrain -= 1
        gnsblood -= 1
    elif inboth == 0:
        gnsnone += 1


print "Percentage of GWAS Catalog NS SNPs Found in Brain Genes:", float(gnsbrain) / gnumnssnps
print "Percentage of GWAS Catalog NS SNPs Found in Blood Genes:", float(gnsblood) / gnumnssnps
print "Percentage of GWAS Catalog NS SNPs Found in Both:", float(gnsboth) / gnumnssnps
print "Percentage of GWAS Catalog NS SNPs Found in Neither:", float(gnsnone) / gnumnssnps


Percentage of GWAS Catalog NS SNPs Found in Brain Genes: 0.0515235457064
Percentage of GWAS Catalog NS SNPs Found in Blood Genes: 0.0470914127424
Percentage of GWAS Catalog NS SNPs Found in Both: 0.0681440443213
Percentage of GWAS Catalog NS SNPs Found in Neither: 0.83324099723


Similarly, the percentages of GWAS Catalog AU SNPs found in brain genes, blood genes, both brain and blood genes, and neither brain nor blood genes are calculated and printed.

In [54]:
# For GWAS Catalog AU SNPs
gaubrain = 0
gaublood = 0
gauboth = 0
gaunone = 0

for snp in gwasausnps:
    snpchr = gwassnpsinfo[snp]["chr"]
    snpbp = gwassnpsinfo[snp]["bp"]

    inbrain = 0
    for start, end in braingenes[snpchr]:
        if inRange(snpbp, start, end):
            inbrain = 1
            break

    inblood = 0
    for start, end in bloodgenes[snpchr]:
        if inRange(snpbp, start, end):
            inblood = 1
            break

    gaubrain += inbrain
    gaublood += inblood

    inboth = inbrain + inblood
    if inboth == 2:
        gauboth += 1
        gaubrain -= 1
        gaublood -= 1
    elif inboth == 0:
        gaunone += 1


print "Percentage of GWAS Catalog AU SNPs Found in Brain Genes:", float(gaubrain) / gnumausnps
print "Percentage of GWAS Catalog AU SNPs Found in Blood Genes:", float(gaublood) / gnumausnps
print "Percentage of GWAS Catalog AU SNPs Found in Both:", float(gauboth) / gnumausnps
print "Percentage of GWAS Catalog AU SNPs Found in Neither:", float(gaunone) / gnumausnps

Percentage of GWAS Catalog AU SNPs Found in Brain Genes: 0.0551420838972
Percentage of GWAS Catalog AU SNPs Found in Blood Genes: 0.0703653585927
Percentage of GWAS Catalog AU SNPs Found in Both: 0.0788227334235
Percentage of GWAS Catalog AU SNPs Found in Neither: 0.795669824087


The specific number of GWAS Catalog NS and AU SNPs in brain, blood, both, and neither are also printed for reference.

In [55]:
print "GWAS Catalog NS SNPs in Brain:", gnsbrain
print "GWAS Catalog NS SNPs in Blood:", gnsblood
print "GWAS Catalog AU SNPs in Brain:", gaubrain
print "GWAS Catalog AU SNPs in Blood:", gaublood

GWAS Catalog NS SNPs in Brain: 93
GWAS Catalog NS SNPs in Blood: 85
GWAS Catalog AU SNPs in Brain: 163
GWAS Catalog AU SNPs in Blood: 208


The Chi-Square value and significance threshold are printed out. If the Chi-Square value is greater than or equal to the significance threshold, then it is indeed significant that GWAS Catalog NS and AU related SNPs are found more in brain or blood genes than a uniform distribution.

In [56]:
# Three Column Chi-Square Test
gnsother = gnsboth + gnsnone
gauother = gauboth + gaunone

gchisquareval = chiSquare(gnsbrain, gnsblood, gaubrain, gaublood, gnsother, gauother)

print "GWAS Catalog Chi Square Value:", gchisquareval
print "Threshold for Significance:", chithreshold

GWAS Catalog Chi Square Value: 11.0530511082
Threshold for Significance: 5.99


# Final Results Output File

The final results of the percentages and numbers of filtered new SNPs list of NS and AU SNPs found in brain genes, blood genes, both, and neither and the Chi-Square test values are printed to a results file. Similarly, the results for the GWAS Catalog NS and AU SNPs are also printed to the results. The file is named "results.txt".

In [57]:
# Writing results to output file
results = open("results.txt", "w")

results.write("Percentage of NS SNPs Found in Brain Genes: " + str(float(nsbrain) / numnssnps) + "\n")
results.write("Percentage of NS SNPs Found in Blood Genes: " + str(float(nsblood) / numnssnps) + "\n")
results.write("Percentage of NS SNPs Found in Both: " + str(float(nsboth) / numnssnps) + "\n")
results.write("Percentage of NS SNPs Found in Neither: " + str(float(nsnone) / numnssnps) + "\n")

results.write("Percentage of AU SNPs Found in Brain Genes: " + str(float(aubrain) / numausnps) + "\n")
results.write("Percentage of AU SNPs Found in Blood Genes: " + str(float(aublood) / numausnps) + "\n")
results.write("Percentage of AU SNPs Found in Both: " + str(float(auboth) / numausnps) + "\n")
results.write("Percentage of AU SNPs Found in Neither: " + str(float(aunone) / numausnps) + "\n")

results.write("Number of NS SNPs Found in Brain Genes: " + str(nsbrain) + "\n")
results.write("Number of NS SNPs Found in Blood Genes: " + str(nsblood) + "\n")

results.write("Number of AU SNPs Found in Brain Genes: " + str(aubrain) + "\n")
results.write("Number of AU SNPs Found in Blood Genes: " + str(aublood) + "\n")

results.write("Chi Square Value for Significance of Non-Uniform Distribution in Brain and Blood Genes: " + str(chisquareval) + "\n")
results.write("Threshold for Significance at 95% Confidence and Two Degrees of Freedom: " + str(chithreshold) +  "\n")

results.write("\n")

results.write("Percentage of GWAS Catalog NS SNPs Found in Brain Genes: " + str(float(gnsbrain) / gnumnssnps) + "\n")
results.write("Percentage of GWAS Catalog NS SNPs Found in Blood Genes: " + str(float(gnsblood) / gnumnssnps) + "\n")
results.write("Percentage of GWAS Catalog NS SNPs Found in Both: " + str(float(gnsboth) / gnumnssnps) + "\n")
results.write("Percentage of GWAS Catalog NS SNPs Found in Neither: " + str(float(gnsnone) / gnumnssnps) + "\n")

results.write("Percentage of GWAS Catalog AU SNPs Found in Brain Genes: " + str(float(gaubrain) / gnumausnps) + "\n")
results.write("Percentage of GWAS Catalog AU SNPs Found in Blood Genes: " + str(float(gaublood) / gnumausnps) + "\n")
results.write("Percentage of GWAS Catalog AU SNPs Found in Both: " + str(float(gauboth) / gnumausnps) + "\n")
results.write("Percentage of GWAS Catalog AU SNPs Found in Neither: " + str(float(gaunone) / gnumausnps) + "\n")

results.write("Number of GWAS Catalog NS SNPs Found in Brain Genes: " + str(gnsbrain) + "\n")
results.write("Number of GWAS Catalog NS SNPs Found in Blood Genes: " + str(gnsblood) + "\n")

results.write("Number of GWAS Catalog AU SNPs Found in Brain Genes: " + str(gaubrain) + "\n")
results.write("Number of GWAS Catalog AU SNPs Found in Blood Genes: " + str(gaublood) + "\n")

results.write("GWAS Catalog Chi Square Value for Significance of Non-Uniform Distribution in Brain and Blood Genes: " + str(gchisquareval) + "\n")
results.write("Threshold for Significance at 95% Confidence and Two Degrees of Freedom: " + str(chithreshold) +  "\n")


results.close()

print "SNP Analysis Complete"

SNP Analysis Complete
